In [ ]:
!pip install altair vl-convert-python

In [9]:
import pandas as pd
from pathlib import Path
import json

files = list(Path('../experiments').glob('**/*scores.json'))

In [11]:
records = []
for path in files:
    with open(path) as f:
        content = json.load(f)
        key = list(content['results'].keys())[0]
        if 'xnli' not in key:
            continue
        scores = content['results'][key]
        scores['n_shots'] = content['n-shot'][key]
        scores['partition'] = path.stem.split('.')[0]
        records.append(scores)
df = pd.DataFrame().from_records(records)

In [12]:
df.head()

,alias,"acc,none","acc_stderr,none","f1,none","f1_stderr,none",n_shots,partition
0,afrixnli_native_direct_yor,0.331667,0.019237,0.269244,N/A,2,eval_pre_filtered
1,afrixnli_native_direct_yor,0.333333,0.019261,0.258951,N/A,8,eval_pre_filtered
2,afrixnli_native_direct_yor,0.311667,0.018925,0.249823,N/A,4,eval_pre_filtered
3,afrixnli_native_direct_yor,0.360000,0.019612,0.278976,N/A,9,eval_pre_filtered
4,afrixnli_native_direct_yor,0.333333,0.019261,0.240593,N/A,1,eval_pre_filtered


In [13]:
import altair as alt

plots = []
for alias in df['alias'].unique():
    temp_plot = (
        alt.Chart(df[df['alias'] == alias], title=f"Lang: {alias.split('_')[-1]}")
        .mark_bar()
        .encode(
            x=alt.X("partition", title=None, axis=alt.Axis(labels=False)),
            y=alt.Y("acc,none", title="Accuracy"),
            column=alt.Column("n_shots", title="N shots"),
            color=alt.Color("partition"),
        )
    )
    plots.append(temp_plot)
final_plot = alt.vconcat(*plots)
final_plot

alt.VConcatChart(...)

In [ ]:
final_plot.save('xnli-results.pdf')